# Plot paired and unpaired synapses

This notebook assumes that all of the studies are orginized into a standard Python data structure. All of the plots are done using plot.ly, so you should look at the appropriate documentation for that if you want to change them.

The variable `study_list` constains a list of studies, where each study is represented as a pyton dictionary. The keys in the dictionary currently include the study name, the subject id, the type of study (aversion, control), and if the fish learned or not. 

The python command `list(study_list[0])` will return a list of the current values that are available for a study.

The study dictionary may also contain a set of calculated synapse pairings.  Each pair is represented as a python dictionary, whose key is the radius with which the pairing was determined, and whose value is a PANDA, with the synapse positions of the pair. Each PANDA has included in it a set of useful values from the study, including the study id, the type of study, the radius for the pair, etc.  Pairs are also transformed using the alignment points to put them into a common coordinate space.  The sets of pairs provided are:

* PairedBefore
* PairedAfter
* UnpairedBefore
* UnpairedAfter
* AlignedPairedBefore
* AlignedPairedAfter
* AlignedUnpairedBefore
* AlignedUnpairedAfter

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Load up all of the libraries we need

import synapse_plot_utils as sp
import synapse_utils
import synapse_plot_config as plot_config
from synapse_plot_config import plot_synapses, trace_color, position_layout, step_buttons, studytypes
from scipy.spatial import ConvexHull, KDTree

import pandas as pd
import numpy as np
import xarray as xr
import plotly as py
import plotly.graph_objs as go
import matplotlib
import csv
import hdbscan

from matplotlib import cm

import colorlover as cl

# configure plot.ly to run in a notebook.
py.offline.init_notebook_mode(connected=True)

# Load Data

In [ ]:
# Points unswapped
fileid = '10H2@2PW-ZBJY-6HT4'
fileid = '1124'
studyid, studyset = synapse_utils.fetch_studies(fileid)
studylist_unswapped = studyset['Studies']

# Points swap
fileid = '112A'
studyid, studyset = synapse_utils.fetch_studies(fileid)
# Local copy
#studyset = synapse_utils.restore_studies('study-10DJ.pkl')


studylist = studyset['Studies']
study_types = studytypes(studylist)

Calculate the distance between synapses.

In [ ]:
rmax = max(studylist[0]['PairedAfter'])
paired_distance  = { k : {'Before':[], 'After':[]} for k in study_types}
unpaired_distance = { k : {'Before':[], 'After':[]} for k in study_types}

# Get all of the synapses in before and after...
for i in studylist:
    for tp in ['Before', 'After']:
        paired = i['Unpaired' + tp][rmax]['Data'][['x','y','z']]
        unpaired = i['Paired' + tp][rmax]['Data'][['x','y','z']]
        before_distance = []
        # Create KD tree for all before points....
        kdtree = KDTree(paired.append(unpaired))
        # Now calculate distance from paired to nearest neigbor
        for idx,pt in paired.iterrows():
            dist, index = kdtree.query(pt,2)
            # Don't include distance to yourself.
            paired_distance[i['Type']][tp].append(dist[1])
            
        # Now calculate the distance from the unpaired to the neareast neighbor
        for idx,pt in unpaired.iterrows():
            dist, index = kdtree.query(pt,2)
            # Don't include distance to yourself.
            unpaired_distance[i['Type']][tp].append(dist[1])

<H1> Compute some basic Statistics

In [ ]:
s = pd.Series(unpaired_distance['learner']['Before']).describe()
s

In [ ]:
# Set what radius/studies you want to have....

cols = ['Synapse Type','Study Type', 'Timepoint',  "Count", 'Mean', 'SD',]
c1, c2, c3, c4, c5, c6 = [],[],[],[],[],[]

syn_pair_radii = list(studylist[0]['UnpairedBefore'])

for t in study_types:
    for i in ['Before', 'After']:
        s = pd.Series(unpaired_distance[t][i]).describe()
        c1.append('unpaired')
        c2.append(t)
        c3.append(i)
        c4.append(s['count'])
        c5.append(s['mean'])
        c6.append(s['std'])
        
        s = pd.Series(paired_distance[t][i]).describe()
        c1.append('paired')
        c2.append(t)
        c3.append(i)
        c4.append(s['count'])
        c5.append(s['mean'])
        c6.append(s['std'])
 
trace = go.Table(
    header=dict(values=cols,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[c1, c2, c3, c4, c5, c6],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))

data = [trace] 
py.offline.iplot(data, filename = 'statistics_table')

In [ ]:
#data_paired  = { k : [] for k in study_types}
#data_unpaired  = { k : [] for k in study_types}

max_distance = 0
for tp in ['Before', 'After']:
    for i in study_types:
        max_distance = max(max_distance, max(paired_distance[i][tp]))
        max_distance = max(max_distance, max(unpaired_distance[i][tp]))
print("Maximum distance is: ", max_distance)

xrange = [0, 10]
histnorm = 'probability density'

data_paired, data_unpaired = [],[]
for tp in ['Before', 'After']:
    data_paired, data_unpaired = [], []
    for i in study_types:
        data_paired.append(go.Histogram(
            histnorm=histnorm,
            opacity=.6,
            x=paired_distance[i][tp], 
            name='{}'.format(i)))
        data_unpaired.append(go.Histogram(
            histnorm=histnorm,
            opacity=.6,
            x=unpaired_distance[i][tp], 
            name='{}'.format(i)))  
    hist_layout = go.Layout(
    title='Synapse Distance Histogram Paired {}'.format(tp),
        xaxis=dict( title='Distance'.format(tp), range=xrange), 
        yaxis=dict(title='Count', range=[0,1]),
        barmode='group',bargap=0.2, bargroupgap=0.1
    )
    hist_fig = go.Figure(data=data_paired, layout=hist_layout)
    py.offline.iplot(hist_fig, filename='pair-distance')
    
    hist_layout = go.Layout(
    title='Synapse Distance Histogram Unpaired {}'.format(tp),
        xaxis=dict( title='Distance'.format(tp), range=xrange), 
        yaxis=dict(title='Count', range=[0,1]),
        barmode='group',bargap=0.2, bargroupgap=0.1
    )
    hist_fig = go.Figure(data=data_unpaired, layout=hist_layout)
    py.offline.iplot(hist_fig, filename='unpair-distance')